# Calculate squares based on the scanpaths


This notebook will calculate squares based on scanpaths for every experience group.
I am going to draw a square around the regions they look at. First it will make a square for every experience group for every scene and image. Next I am going to make a square for every group on one picture with different colors.


In [ ]:
from matplotlib import image 
from matplotlib import pyplot as plt 
import pandas as pd
import cv2
import numpy as np
import matplotlib.patches as patches
from PIL import Image

import os
root = '../../heatmaps/squares/'

pad = "../../data/csv/"
image_pad =  "../../data/Images/CrimeScenes/"

df = pd.read_csv(pad + 'fix_evidence.csv')

We take the min and max values so that we can create our borders.


In [2]:
df.groupby(['experience', 'Scene', 'Image']).agg({'X': ['min', 'max'],'Y': ['min', 'max']}).head()

X             Y       
                         min     max   min    max
experience Scene Image                           
CSI        1     1      55.3   730.2   7.7  767.6
                 2       0.2  1004.6  13.9  750.1
           2     1       1.0   976.7   7.9  759.9
                 2       1.6   991.9   3.0  752.8
           3     1      37.9  1010.5  10.5  762.4

In [3]:
def load_image(img_pad, scene_nr, image_nr):
    maxx,maxy = 1024, 768
    img = cv2.imread(img_pad)
    height,width, colours = img.shape
    bg = 255*np.ones((maxy,maxx,3),dtype=np.uint8)
    startX, startY = int((maxx-width)/2), int((maxy-height)/2)
    print(scene_nr, image_nr, height, startY)
    if scene_nr == 1 and image_nr == 1:
        startX = 44 #value found in the Matlab code which must have been based on the comparison with the Eyelink viewer
        #startY = 0
    #bg[startY:startY+height, startX:startX+width,:] = img
    bg[startY:startY + height, startX:startX+width,:] = img
    return(bg)

In [4]:
def image_square(exp, image, scene):
    
    subfolder = f'squares/scene{scene}_{image}/'
    os.makedirs(root+subfolder, exist_ok=True)    
    
    im = Image.open(image_pad + f'scene_{scene}_{image}.png')
    #im = load_image(image_pad + f'scene_{scene}_{image}.png', scene, image)
    
    # Create figure and axes
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(im)
    # Create a Rectangle patch
    df_img = df[(df['Image'] == image) & (df['Scene'] == scene)]
    df_grouped = df_img.groupby(['experience']).agg({'X': ['min', 'max'],'Y': ['min', 'max']}).reset_index()
    x = df_grouped[df_grouped['experience'] == exp]['X']['min'].item()
    y = df_grouped[df_grouped['experience'] == exp]['Y']['min'].item()
    w = df_grouped[df_grouped['experience'] == exp]['X']['max'].item() - x
    h = df_grouped[df_grouped['experience'] == exp]['Y']['max'].item() - y
    
    print(x, y, w, h)


    rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)
#     plt.xlim([0, 1024])
#     plt.ylim([0, 768])

    plt.draw()
    plt.savefig(f'{root}{subfolder}heatmap_{exp}.png', bbox_inches='tight', pad_inches=0)
    
    plt.close(fig)

In [5]:
scenes = [1, 2, 3]
images = [1, 2]
experiences = ['CSI', 'Control', 'FirstYear', 'ThirdYear']

for image in images:
    for scene in scenes:
         for exp in experiences:
            image_square(exp, image, scene)


55.3 7.7 674.9000000000001 759.9
58.0 2.6 672.9 760.6
13.3 35.7 716.6 730.4
47.7 57.0 682.9 710.4
1.0 7.9 975.7 752.0
4.6 1.6 998.6 762.9
3.5 2.0 975.0 747.4
0.9 4.8 1004.3000000000001 758.0
37.9 10.5 972.6 751.9
28.0 1.9 967.4 751.4
64.0 46.4 884.9 687.8000000000001
18.3 2.4 1003.2 754.5
0.2 13.9 1004.4 736.2
2.1 4.5 996.1999999999999 742.6
3.0 11.3 996.1 742.7
19.0 6.0 967.6 760.3
1.6 3.0 990.3 749.8
14.9 2.2 1000.1 764.0
16.8 119.5 969.5 642.7
15.9 3.4 990.3000000000001 754.9
3.3 10.8 1012.8000000000001 753.9000000000001
27.2 5.2 994.4 760.9
47.2 18.7 956.6999999999999 737.5999999999999
12.7 1.2 1010.3 762.0999999999999


Make squares within one picture and every experience group an other color.


In [6]:
def image_square_color(image, scene):
    
    experiences = ['CSI', 'Control', 'FirstYear', 'ThirdYear']
    colors = ['r', 'g', 'b', 'm']
    subfolder = f'squares/scene{scene}_{image}/'
    os.makedirs(root+subfolder, exist_ok=True)    
    
    im = Image.open(image_pad + f'scene_{scene}_{image}.png')
    #im = load_image(image_pad + f'scene_{scene}_{image}.png', scene, image)

    # Create figure and axes
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(im)

    # Create a Rectangle patch
    df_img = df[(df['Image'] == image) & (df['Scene'] == scene)]
    df_grouped = df_img.groupby(['experience']).agg({'X': ['min', 'max'],'Y': ['min', 'max']}).reset_index()
    for i in range(len(experiences)):
        x = df_grouped[df_grouped['experience'] == experiences[i]]['X']['min'].item()
        y = df_grouped[df_grouped['experience'] == experiences[i]]['Y']['min'].item()
        w = df_grouped[df_grouped['experience'] == experiences[i]]['X']['max'].item() - x
        h = df_grouped[df_grouped['experience'] == experiences[i]]['Y']['max'].item() - y

        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor=colors[i], facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)

    plt.draw()
    plt.savefig(f'{root}{subfolder}heatmap.png', bbox_inches='tight', pad_inches=0)
    
    plt.close(fig)

In [7]:
scenes = [1, 2, 3]
images = [1, 2]

for image in images:
    for scene in scenes:
        image_square_color(image, scene)
